# <div align="center">Detail understanding of SMPL paper</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


### <div align="center">Training</div>

We train the SMPL model parameters to minimize reconstruction error on two datasets. Each dataset contains meshes with the same topology as our template that have been aligned to high-resolution 3D scans; we call these aligned meshes ***registrations***. The ***multi-pose*** dataset consists of 1786 registrations of ***40 individuals*** (891 registrations spanning 20 females, and 895 registrations spanning 20 males); a sampling is shown in figure below. The ***multi-shape*** dataset consists of registrations to the CAESAR dataset, totaling 1700 registrations for males and 2100 for females; a few examples are shown in Figure below. We denote the j-th mesh in the multi-pose dataset as $V_j^P$ and the j-th mesh in the multi-shape dataset as $V_j^S$.

<img src='_assets/13.png'>

Our goal is to train the parameters $\mathcal{\phi} = \{ \bar{T}, \mathcal{W}, \mathcal{S}, \mathcal{J}, \mathcal{P} \}$ to minimize ***vertex reconstruction error*** on the datasets. Because our model decomposes shape and pose, we train these separately, simplifying optimization. We first train $\{ \mathcal{W}, \mathcal{J}, \mathcal{P} \}$ using our multi-pose
dataset and then train $\{ \bar{T}, \mathcal{S} \}$ using our multi-shape dataset. We
train separate models for men and women (i.e. $\mathcal{\Phi}_m$ and $\mathcal{\Phi}_f$ ).

### <div align="center">Pose Parameter Training</div>

We first use the multi-pose dataset to train $\{ \mathcal{W}, \mathcal{J}, \mathcal{P} \}$. To this end, we need to compute the rest templates, $\hat{T}_i^P$ , and joint locations, $\hat{J}_i^P$, for each subject, $i$, as well as the pose parameters, $\theta_j$, for each registration, $j$, in the dataset. We alternate between optimizing 

* registration specific parameters $\theta_j$

* subject-specific parameters $\{ \hat{T}_i^P, \hat{J}_i^P \}$

* global parameters $\{ \mathcal{W}, \mathcal{P} \}$, and

* matrix $\mathcal{J}$, to regress from subject-specific vertex locations, $\hat{T}_i^P$, to subject-specific joint locations, $\hat{J}_i^P$.

To achieve all this, we minimize an objective function consisting of a data term, $E_D$, and several regularization terms $\{ E_J, E_Y, E_P, E_W \}$ defined below.

The data term penalizes the squared Euclidean distance between registration vertices and model vertices

<div align="center">$E_D(\hat{T}^P, \hat{J}^P, \mathcal{W}, \mathcal{P}, \Theta) = \displaystyle{\sum_{j=1}^{P_{reg}}}||V_j^P - W(\hat{T}_{s(j)}^P + B_P(\theta_j, \mathcal{P}), \hat{J}_{s(j)}^P, \theta_j, \mathcal{W})||^2$</div>

where

* $\Theta = \{ \theta_1,..., \theta_{P_{reg}} \}$, $s(j)$ is the subject index corresponding to registration $j$

* $P_{reg}$ is the number of meshes in the pose trainings set

* $\hat{T}^P$ and $\hat{J}^P$ are the sets of all ***rest poses*** and ***joints***

We estimate 
* 207 * 3 * 6890 = 4.278.690 parameters for the pose blend shapes, $\mathcal{P}$ 

* 4 * 3 * 6890 = 82.680 parameters for the skinning weights, $\mathcal{W}$, and 

* 3 * 6890 * 23 * 3 = 1.426.230 for the joint regressor matrix, $\mathcal{J}$. 

To make the estimation well behaved, we regularize by making several assumptions.

***Symmetry regularization term, $E_Y$*** penalizes left-right asymmetry for $\hat{J}^P$ and $\hat{T}^P$

<div align="center">$E_Y(\hat{J}^P, \hat{T}^P) = \displaystyle{\sum_{i=1}^{P_{subj}}} \lambda_U ||\hat{J}_i^P - U(\hat{J}_i^P)||^2 + ||\hat{T}_i^P - U(\hat{T}_i^P)||^2$</div>

where

* $\lambda_U$ = 100 and

* $U(T)$ finds a mirror image of vertices T, by flipping across the sagittal plane and swapping symmetric vertices.

This term encourages ***symmetric template meshes*** and, more importantly, ***symmetric joint locations***. Joints are unobserved variables and along the spine they are particularly difficult to localize. While models trained without the symmetry term produce reasonable results, enforcing symmetry produces models that are visually more intuitive for animation.



Our model is hand segmented into 24 parts. We use this segmentation to compute an initial estimate of the joint centers and
a regressor $\mathcal{J_I}$ from vertices to these centers. This regressor computes the initial joints by taking the average of the ring of vertices connecting two parts. When estimating the joints for each subject we ***regularize them to be close to this initial prediction***:

<div align="center">$E_J(\hat{J}^P, \hat{T}^P) = \displaystyle{\sum_{i=1}^{P_{subj}}} ||\mathcal{J}_I \hat{T}_i^P - \hat{J}_i^P||^2$</div>

To help prevent overfitting of the pose-dependent blend shapes, we ***regularize them towards zero***

<div align="center">$E_P(\mathcal{P})=||\mathcal{P}||_F^2$</div>

where 
* $||\cdot||_F$ denotes the Frobenius norm. 

Note that replacing the quadratic penalty with an L1 penalty would encourage greater sparsity of the blend shapes. We did not try this.

We also ***regularize the blend weights*** towards the initial weights, $\mathcal{W}_I$

<div align="center">$E_W(\mathcal{W})=||\mathcal{W} - \mathcal{W}_I||_F^2$</div>

The initial weights are computed by simply diffusing the segmentation.

Altogether, the energy for training $\{ \mathcal{W}, \mathcal{P} \}$ is as follows:

<div align="center">$E_*(\hat{T}^P, \hat{J}^P, \mathcal{W}, \mathcal{P}, \Theta) = E_D + \lambda_Y E_Y + \lambda_J E_J + \lambda_P E_P + E_W$</div>

where 

* $\lambda_Y$ = 100, $\lambda_J$ = 100 and $\lambda_P$ = 25. These weights were set empirically. 

### Joint regressor.
Optimizing the above gives a template mesh and joint locations for each subject, but we want to predict joint locations for new subjects with new body shapes. To that end, we learn the regressor matrix $\mathcal{J}$ to predict the training joints from the
training bodies. We tried several regression strategies; what we found to work best, was to compute $\mathcal{J}$ using non-negative least squares [Lawson and Hanson 1995] with the inclusion of a term that encourages the weights to add to one. This approach encourages sparsity of the vertices used to predict the joints. Making weights positive and add to one discourages predicting joints outside the surface. These constraints enforce the predictions to be in the convex hull of surface points.

<img src='_assets/14.png'>